# Notebook 9: Resolution de Sudoku par Coloration de Graphe

[< Retour a l'index](./README.md) | **Notebook precedent**: [8 - Strategies Humaines](./Sudoku-8-HumanStrategies.ipynb) | **Notebook suivant**: [10 - OR-Tools](./Sudoku-10-ORTools.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
- **Modeliser** un Sudoku comme un probleme de coloration de graphe
- **Comprendre** la theorie des graphes sous-jacente (81 sommets, degre 20)
- **Implementer** les algorithmes de coloration (Backtracking, DSATUR)
- **Comparer** l'efficacite de différentes strategies de coloration

**Duree estimee** : 30-40 minutes
**Prerequis** : Notebook 0 (Environment), bases de theorie des graphes

## Introduction : Sudoku et Theorie des Graphes

Le Sudoku peut etre modelise comme un **probleme de coloration de graphe** :

### Modelisation
- **Sommets** : 81 cellules de la grille (9x9)
- **Aretes** : deux cellules sont reliees si elles ne peuvent pas avoir la meme valeur
  - Meme ligne (8 voisins par ligne)
  - Meme colonne (8 voisins par colonne)
  - Meme bloc 3x3 (4 voisins supplementaires, car 8-4 sont deja comptes)
- **Couleurs** : valeurs 1 a 9

### Proprietes du graphe
- **Nombre de sommets** : 81
- **Degre de chaque sommet** : 20 (8 + 8 + 4)
- **Nombre d'aretes** : 81 * 20 / 2 = 810
- **Clique maximale** : 9 (ligne, colonne ou bloc complet)

Cette formulation permet d'appliquer des algorithmes classiques de coloration !

In [ ]:
// Import des classes de base
#!import "Sudoku-0-Environment-Csharp.ipynb"

## Construction du Graphe Sudoku

Nous definissons une classe `SudokuGraph` qui represente le graphe d'un Sudoku :
- Chaque sommet correspond a une cellule (index 0-80)
- Les aretes representent les contraintes d'exclusion
- La coloration (assignation de valeurs) doit respecter les aretes

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;

/// <summary>
/// Represente le graphe de contraintes d'un Sudoku.
/// 81 sommets (cellules), aretes entre cellules en conflit.
/// </summary>
public class SudokuGraph
{
    // Nombre de sommets (81 cellules)
    public const int VertexCount = 81;
    
    // Nombre de couleurs (valeurs 1-9)
    public const int ColorCount = 9;
    
    // Liste d'adjacence : adjacency[v] = liste des voisins du sommet v
    private readonly List<int>[] _adjacency;
    
    // Degre de chaque sommet
    private readonly int[] _degrees;
    
    // Grille Sudoku associee
    private readonly SudokuGrid _grid;
    
    // Cellules pre-coloriees (indices fixes)
    private readonly HashSet<int> _fixedVertices;
    
    /// <summary>
    /// Construit le graphe a partir d'une grille Sudoku.
    /// </summary>
    public SudokuGraph(SudokuGrid grid)
    {
        _grid = grid;
        _adjacency = new List<int>[VertexCount];
        _degrees = new int[VertexCount];
        _fixedVertices = new HashSet<int>();
        
        // Initialiser les listes d'adjacence
        for (int i = 0; i < VertexCount; i++)
        {
            _adjacency[i] = new List<int>();
        }
        
        // Construire les aretes a partir des contraintes Sudoku
        BuildEdges();
        
        // Identifier les cellules pre-coloriees
        IdentifyFixedVertices();
    }
    
    /// <summary>
    /// Construit les aretes du graphe.
    /// Chaque cellule est reliee a ses voisins de ligne, colonne et bloc.
    /// </summary>
    private void BuildEdges()
    {
        for (int row = 0; row < 9; row++)
        {
            for (int col = 0; col < 9; col++)
            {
                int vertex = ToVertexIndex(row, col);
                var neighbors = GetNeighborVertices(row, col);
                
                foreach (var neighbor in neighbors)
                {
                    if (!_adjacency[vertex].Contains(neighbor))
                    {
                        _adjacency[vertex].Add(neighbor);
                        _degrees[vertex]++;
                    }
                }
            }
        }
    }
    
    /// <summary>
    /// Obtient tous les sommets voisins d'une cellule.
    /// </summary>
    private HashSet<int> GetNeighborVertices(int row, int col)
    {
        var neighbors = new HashSet<int>();
        
        // Voisins de ligne
        for (int c = 0; c < 9; c++)
            if (c != col)
                neighbors.Add(ToVertexIndex(row, c));
        
        // Voisins de colonne
        for (int r = 0; r < 9; r++)
            if (r != row)
                neighbors.Add(ToVertexIndex(r, col));
        
        // Voisins de bloc 3x3
        int blockRow = (row / 3) * 3;
        int blockCol = (col / 3) * 3;
        for (int r = blockRow; r < blockRow + 3; r++)
        {
            for (int c = blockCol; c < blockCol + 3; c++)
            {
                if (r != row || c != col)
                    neighbors.Add(ToVertexIndex(r, c));
            }
        }
        
        return neighbors;
    }
    
    /// <summary>
    /// Identifie les cellules pre-coloriees (valeurs fixees).
    /// </summary>
    private void IdentifyFixedVertices()
    {
        for (int row = 0; row < 9; row++)
        {
            for (int col = 0; col < 9; col++)
            {
                if (_grid.Cells[row, col] > 0)
                {
                    _fixedVertices.Add(ToVertexIndex(row, col));
                }
            }
        }
    }
    
    // Conversion coordonnees <-> index de sommet
    public static int ToVertexIndex(int row, int col) => row * 9 + col;
    public static (int row, int col) ToCoordinates(int vertex) => (vertex / 9, vertex % 9);
    
    // Acces aux proprietes du graphe
    public IReadOnlyList<int> GetNeighbors(int vertex) => _adjacency[vertex];
    public int GetDegree(int vertex) => _degrees[vertex];
    public bool IsFixed(int vertex) => _fixedVertices.Contains(vertex);
    public int GetInitialColor(int vertex)
    {
        var (row, col) = ToCoordinates(vertex);
        return _grid.Cells[row, col];
    }
    
    /// <summary>
    /// Obtient les couleurs disponibles pour un sommet.
    /// </summary>
    public HashSet<int> GetAvailableColors(int vertex, int[] coloring)
    {
        var usedColors = new HashSet<int>();
        
        foreach (var neighbor in _adjacency[vertex])
        {
            if (coloring[neighbor] > 0)
                usedColors.Add(coloring[neighbor]);
        }
        
        var available = new HashSet<int>();
        for (int color = 1; color <= ColorCount; color++)
        {
            if (!usedColors.Contains(color))
                available.Add(color);
        }
        
        return available;
    }
    
    /// <summary>
    /// Verifie si une coloration est valide.
    /// </summary>
    public bool IsValidColoring(int[] coloring)
    {
        for (int v = 0; v < VertexCount; v++)
        {
            if (coloring[v] == 0) continue;
            
            foreach (var neighbor in _adjacency[v])
            {
                if (coloring[neighbor] == coloring[v])
                    return false;
            }
        }
        return true;
    }
    
    /// <summary>
    /// Affiche les statistiques du graphe.
    /// </summary>
    public void PrintStatistics()
    {
        int totalEdges = _degrees.Sum() / 2;
        int maxDegree = _degrees.Max();
        int minDegree = _degrees.Min();
        
        Console.WriteLine("=== Statistiques du Graphe Sudoku ===");
        Console.WriteLine($"Sommets: {VertexCount}");
        Console.WriteLine($"Aretes: {totalEdges}");
        Console.WriteLine($"Degre min: {minDegree}, max: {maxDegree}");
        Console.WriteLine($"Cellules fixees: {_fixedVertices.Count}");
        Console.WriteLine($"Cellules a colorier: {VertexCount - _fixedVertices.Count}");
    }
}

### Interpretation

La classe `SudokuGraph` encapsule la structure du probleme :
- **81 sommets** representent les 81 cellules
- **810 aretes** (sans doublons) representent les contraintes
- Chaque sommet a un **degre 20** (8 ligne + 8 colonne + 4 bloc supplementaires)
- Les **couleurs disponibles** sont calculees dynamiquement selon l'etat actuel

## Algorithme 1 : Coloration par Backtracking Simple

L'approche naive colorie les sommets sequentiellement :
1. Choisir le premier sommet non colorie
2. Essayer chaque couleur disponible
3. Si conflit, revenir en arriere (backtrack)
4. Repeter jusqu'a coloration complete

In [ ]:
/// <summary>
/// Solveur par coloration de graphe avec backtracking simple.
/// </summary>
public class GraphColoringBacktracking : ISudokuSolver
{
    private int _nodesExplored;
    private int _backtracks;
    
    public int NodesExplored => _nodesExplored;
    public int Backtracks => _backtracks;
    
    public SudokuGrid Solve(SudokuGrid grid)
    {
        _nodesExplored = 0;
        _backtracks = 0;
        
        var graph = new SudokuGraph(grid);
        var coloring = new int[SudokuGraph.VertexCount];
        
        // Initialiser avec les valeurs pre-existantes
        for (int v = 0; v < SudokuGraph.VertexCount; v++)
        {
            coloring[v] = graph.GetInitialColor(v);
        }
        
        // Lancer le backtracking
        if (BacktrackColor(graph, coloring, 0))
        {
            return ColoringToGrid(coloring, grid);
        }
        
        return grid; // Echec
    }
    
    /// <summary>
    /// Backtracking recursif pour la coloration.
    /// </summary>
    private bool BacktrackColor(SudokuGraph graph, int[] coloring, int vertex)
    {
        _nodesExplored++;
        
        // Trouver le prochain sommet a colorier
        while (vertex < SudokuGraph.VertexCount)
        {
            if (coloring[vertex] == 0) break; // Sommet non colorie
            vertex++;
        }
        
        // Tous les sommets sont colories
        if (vertex >= SudokuGraph.VertexCount)
            return true;
        
        // Obtenir les couleurs disponibles
        var availableColors = graph.GetAvailableColors(vertex, coloring);
        
        // Essayer chaque couleur
        foreach (var color in availableColors)
        {
            coloring[vertex] = color;
            
            if (BacktrackColor(graph, coloring, vertex + 1))
                return true;
            
            // Backtrack
            coloring[vertex] = 0;
            _backtracks++;
        }
        
        return false; // Aucune couleur possible
    }
    
    /// <summary>
    /// Convertit une coloration en grille Sudoku.
    /// </summary>
    private SudokuGrid ColoringToGrid(int[] coloring, SudokuGrid original)
    {
        var result = (SudokuGrid)original.Clone();
        for (int v = 0; v < SudokuGraph.VertexCount; v++)
        {
            var (row, col) = SudokuGraph.ToCoordinates(v);
            result.Cells[row, col] = coloring[v];
        }
        return result;
    }
}

### Test du Backtracking Simple

In [ ]:
// Test avec un Sudoku facile
var easySudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).First();
Console.WriteLine($"Sudoku original:\n{easySudoku}");

var solver = new GraphColoringBacktracking();
var stopwatch = System.Diagnostics.Stopwatch.StartNew();
var solved = solver.Solve(easySudoku);
stopwatch.Stop();

Console.WriteLine($"\nSudoku resolu:\n{solved}");
Console.WriteLine($"\nStatistiques:");
Console.WriteLine($"  Noeuds explores: {solver.NodesExplored}");
Console.WriteLine($"  Backtracks: {solver.Backtracks}");
Console.WriteLine($"  Temps: {stopwatch.Elapsed.TotalMilliseconds:F2} ms");
Console.WriteLine($"  Valide: {solved.IsValid(easySudoku)}");

## Algorithme 2 : Coloration avec Heuristique MRV

L'heuristique **MRV (Minimum Remaining Values)** choisit le sommet avec le moins de couleurs disponibles. Cela reduit l'arbre de recherche en detectant les echecs plus tot.

In [ ]:
/// <summary>
/// Solveur par coloration avec heuristique MRV.
/// Choisit le sommet avec le moins de couleurs disponibles.
/// </summary>
public class GraphColoringMRV : ISudokuSolver
{
    private int _nodesExplored;
    private int _backtracks;
    
    public int NodesExplored => _nodesExplored;
    public int Backtracks => _backtracks;
    
    public SudokuGrid Solve(SudokuGrid grid)
    {
        _nodesExplored = 0;
        _backtracks = 0;
        
        var graph = new SudokuGraph(grid);
        var coloring = new int[SudokuGraph.VertexCount];
        
        for (int v = 0; v < SudokuGraph.VertexCount; v++)
        {
            coloring[v] = graph.GetInitialColor(v);
        }
        
        if (BacktrackMRV(graph, coloring))
        {
            return ColoringToGrid(coloring, grid);
        }
        
        return grid;
    }
    
    /// <summary>
    /// Selectionne le sommet avec le moins de couleurs disponibles (MRV).
    /// </summary>
    private int SelectVertexMRV(SudokuGraph graph, int[] coloring)
    {
        int bestVertex = -1;
        int minColors = int.MaxValue;
        
        for (int v = 0; v < SudokuGraph.VertexCount; v++)
        {
            if (coloring[v] != 0) continue; // Deja colorie
            
            var available = graph.GetAvailableColors(v, coloring);
            if (available.Count < minColors)
            {
                minColors = available.Count;
                bestVertex = v;
            }
        }
        
        return bestVertex;
    }
    
    private bool BacktrackMRV(SudokuGraph graph, int[] coloring)
    {
        _nodesExplored++;
        
        // Choisir le sommet avec MRV
        int vertex = SelectVertexMRV(graph, coloring);
        
        // Tous les sommets sont colories
        if (vertex == -1)
            return true;
        
        var availableColors = graph.GetAvailableColors(vertex, coloring);
        
        // Echec si aucune couleur disponible
        if (availableColors.Count == 0)
            return false;
        
        foreach (var color in availableColors)
        {
            coloring[vertex] = color;
            
            if (BacktrackMRV(graph, coloring))
                return true;
            
            coloring[vertex] = 0;
            _backtracks++;
        }
        
        return false;
    }
    
    private SudokuGrid ColoringToGrid(int[] coloring, SudokuGrid original)
    {
        var result = (SudokuGrid)original.Clone();
        for (int v = 0; v < SudokuGraph.VertexCount; v++)
        {
            var (row, col) = SudokuGraph.ToCoordinates(v);
            result.Cells[row, col] = coloring[v];
        }
        return result;
    }
}

## Algorithme 3 : DSATUR (Degree of Saturation)

**DSATUR** est un algorithme de coloration optimal pour les graphes. Il utilise :
- **Degre de saturation** : nombre de couleurs differentes dans le voisinage
- **Tie-breaker** : choisir le sommet avec le plus haut degre en cas d'egalite

Cette heuristique est particulierement efficace pour les graphes reguliers comme le Sudoku.

In [ ]:
/// <summary>
/// Solveur DSATUR (Degree of Saturation).
/// Heuristique optimale pour la coloration de graphes.
/// </summary>
public class GraphColoringDSATUR : ISudokuSolver
{
    private int _nodesExplored;
    private int _backtracks;
    
    public int NodesExplored => _nodesExplored;
    public int Backtracks => _backtracks;
    
    public SudokuGrid Solve(SudokuGrid grid)
    {
        _nodesExplored = 0;
        _backtracks = 0;
        
        var graph = new SudokuGraph(grid);
        var coloring = new int[SudokuGraph.VertexCount];
        
        for (int v = 0; v < SudokuGraph.VertexCount; v++)
            coloring[v] = graph.GetInitialColor(v);
        
        if (BacktrackDSATUR(graph, coloring))
            return ColoringToGrid(coloring, grid);
        
        return grid;
    }
    
    /// <summary>
    /// Calcule le degre de saturation d'un sommet.
    /// Saturation = nombre de couleurs differentes dans le voisinage.
    /// </summary>
    private int CalculateSaturation(SudokuGraph graph, int vertex, int[] coloring)
    {
        var neighborColors = new HashSet<int>();
        
        foreach (var neighbor in graph.GetNeighbors(vertex))
        {
            if (coloring[neighbor] > 0)
                neighborColors.Add(coloring[neighbor]);
        }
        
        return neighborColors.Count;
    }
    
    /// <summary>
    /// Selectionne le sommet avec la plus haute saturation.
    /// Tie-breaker : plus haut degre.
    /// </summary>
    private int SelectVertexDSATUR(SudokuGraph graph, int[] coloring)
    {
        int bestVertex = -1;
        int maxSaturation = -1;
        int maxDegree = -1;
        
        for (int v = 0; v < SudokuGraph.VertexCount; v++)
        {
            if (coloring[v] != 0) continue;
            
            int saturation = CalculateSaturation(graph, v, coloring);
            int degree = graph.GetDegree(v);
            
            // Comparaison : maximiser saturation, puis degre
            if (saturation > maxSaturation || 
                (saturation == maxSaturation && degree > maxDegree))
            {
                maxSaturation = saturation;
                maxDegree = degree;
                bestVertex = v;
            }
        }
        
        return bestVertex;
    }
    
    private bool BacktrackDSATUR(SudokuGraph graph, int[] coloring)
    {
        _nodesExplored++;
        
        int vertex = SelectVertexDSATUR(graph, coloring);
        
        if (vertex == -1)
            return true;
        
        var availableColors = graph.GetAvailableColors(vertex, coloring);
        
        if (availableColors.Count == 0)
            return false;
        
        // Trier les couleurs par frequence decroissante (heuristique LCV)
        var sortedColors = availableColors
            .OrderByDescending(c => CountColorUsage(c, graph, coloring))
            .ToList();
        
        foreach (var color in sortedColors)
        {
            coloring[vertex] = color;
            
            if (BacktrackDSATUR(graph, coloring))
                return true;
            
            coloring[vertex] = 0;
            _backtracks++;
        }
        
        return false;
    }
    
    /// <summary>
    /// Compte l'utilisation d'une couleur dans le voisinage.
    /// </summary>
    private int CountColorUsage(int color, SudokuGraph graph, int[] coloring)
    {
        int count = 0;
        for (int v = 0; v < SudokuGraph.VertexCount; v++)
        {
            if (coloring[v] == color)
                count++;
        }
        return count;
    }
    
    private SudokuGrid ColoringToGrid(int[] coloring, SudokuGrid original)
    {
        var result = (SudokuGrid)original.Clone();
        for (int v = 0; v < SudokuGraph.VertexCount; v++)
        {
            var (row, col) = SudokuGraph.ToCoordinates(v);
            result.Cells[row, col] = coloring[v];
        }
        return result;
    }
}

## Benchmark : Comparaison des Algorithmes

In [ ]:
using System.Collections.Generic;

/// <summary>
/// Resultat d'un test de solveur.
/// </summary>
public record SolverResult(
    string Name,
    bool Success,
    double TimeMs,
    int NodesExplored,
    int Backtracks
);

/// <summary>
/// Teste plusieurs solveurs de coloration.
/// </summary>
public List<SolverResult> BenchmarkSolvers(List<SudokuGrid> sudokus, int limit = 5)
{
    var results = new List<SolverResult>();
    
    var solvers = new List<(string Name, Func<ISudokuSolver> Factory, Func<ISudokuSolver, int> GetNodes, Func<ISudokuSolver, int> GetBacktracks)>
    {
        ("Backtracking Simple", () => new GraphColoringBacktracking(), s => ((GraphColoringBacktracking)s).NodesExplored, s => ((GraphColoringBacktracking)s).Backtracks),
        ("MRV Heuristic", () => new GraphColoringMRV(), s => ((GraphColoringMRV)s).NodesExplored, s => ((GraphColoringMRV)s).Backtracks),
        ("DSATUR", () => new GraphColoringDSATUR(), s => ((GraphColoringDSATUR)s).NodesExplored, s => ((GraphColoringDSATUR)s).Backtracks)
    };
    
    foreach (var (name, factory, getNodes, getBacktracks) in solvers)
    {
        int successCount = 0;
        long totalTime = 0;
        int totalNodes = 0;
        int totalBacktracks = 0;
        
        foreach (var sudoku in sudokus.Take(limit))
        {
            var solver = factory();
            var sw = System.Diagnostics.Stopwatch.StartNew();
            var solved = solver.Solve(sudoku);
            sw.Stop();
            
            if (solved.IsValid(sudoku))
            {
                successCount++;
                totalTime += sw.ElapsedMilliseconds;
                totalNodes += getNodes(solver);
                totalBacktracks += getBacktracks(solver);
            }
        }
        
        results.Add(new SolverResult(
            name,
            successCount == limit,
            successCount > 0 ? (double)totalTime / successCount : -1,
            totalNodes / Math.Max(1, successCount),
            totalBacktracks / Math.Max(1, successCount)
        ));
    }
    
    return results;
}

In [ ]:
// Execution du benchmark
Console.WriteLine("=== Benchmark : Coloration de Graphe pour Sudoku ===\n");

// Test sur Sudokus faciles
var easySudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Easy);
Console.WriteLine($"Test sur {Math.Min(5, easySudokus.Count)} Sudokus faciles...\n");

var easyResults = BenchmarkSolvers(easySudokus, 5);

Console.WriteLine("| Algorithme | Succes | Temps Moy (ms) | Noeuds | Backtracks |");
Console.WriteLine("|------------|--------|----------------|--------|------------|");
foreach (var r in easyResults)
{
    Console.WriteLine($"| {r.Name,-20} | {(r.Success ? "OK" : "ECHEC")} | {r.TimeMs,14:F2} | {r.NodesExplored,6} | {r.Backtracks,10} |");
}

In [ ]:
// Test sur Sudokus difficiles
var hardSudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Hard);
Console.WriteLine($"\nTest sur {Math.Min(3, hardSudokus.Count)} Sudokus difficiles...\n");

var hardResults = BenchmarkSolvers(hardSudokus, 3);

Console.WriteLine("| Algorithme | Succes | Temps Moy (ms) | Noeuds | Backtracks |");
Console.WriteLine("|------------|--------|----------------|--------|------------|");
foreach (var r in hardResults)
{
    Console.WriteLine($"| {r.Name,-20} | {(r.Success ? "OK" : "ECHEC")} | {r.TimeMs,14:F2} | {r.NodesExplored,6} | {r.Backtracks,10} |");
}

### Analyse des Resultats

| Algorithme | Complexite | Avantages | Inconvenients |
|------------|------------|-----------|---------------|
| **Backtracking Simple** | O(9^81) theorique | Simple a implementer | Très lent sans heuristique |
| **MRV** | O(9^n) avec n < 81 | Detecte echecs rapidement | Ordonnancement statique |
| **DSATUR** | O(n^2) par iteration | Optimal pour graphes reguliers | Plus complexe |

**Observations typiques** :
- DSATUR reduit significativement le nombre de backtracks
- MRV offre un bon compromis simplicite/efficacite
- La difference est plus marquee sur les Sudokus difficiles

## Integration au Framework Sudoku

In [ ]:
// Test d'integration avec SudokuHelper
Console.WriteLine("=== Test d'integration avec SudokuHelper ===\n");

var dsaturSolver = new GraphColoringDSATUR();
SudokuHelper.SolveSudoku(easySudokus.First(), dsaturSolver);

## Exercices

### Exercice 1 : Coloration Gloutonne
Implementez un algorithme glouton qui colorie les sommets dans l'ordre sans backtracking :
```csharp
public class GraphColoringGreedy : ISudokuSolver { ... }
```
**Question** : Pourquoi cette approche ne peut-elle pas resoudre un Sudoku ?

### Exercice 2 : Propagation de Contraintes
Ajoutez une etape de propagation apres chaque assignation :
- Si un voisin n'a plus qu'une couleur disponible, l'assigner
- Propager recursivement jusqu'a stabilite

### Exercice 3 : Visualisation du Graphe
Utilisez Plotly.NET pour visualiser le graphe de coloration avec les couleurs assignees.

**Indice** : Utilisez un layout circulaire pour les 81 sommets.

## Conclusion

Dans ce notebook, nous avons :

1. **Modelise** le Sudoku comme un probleme de coloration de graphe (81 sommets, degre 20)
2. **Implemente** trois algorithmes de coloration :
   - Backtracking simple (baseline)
   - MRV (Minimum Remaining Values)
   - DSATUR (Degree of Saturation)
3. **Compare** leurs performances sur differentes difficultes

### Connexions avec d'autres approches

| Notebook | Lien conceptuel |
|----------|-----------------|
| [Sudoku-6-AIMA-CSP](./Sudoku-6-AIMA-CSP.ipynb) | CSP = coloration avec variables/domaines/contraintes |
| [Sudoku-7-Norvig](./Sudoku-7-Norvig.ipynb) | Propagation similaire a la reduction de domaine |
| [Search-6-CSP-Fundamentals](../Search/Foundations/Search-6-CSP-Fundamentals.ipynb) | Theorie generale des CSP |

---

[< Retour a l'index](./README.md) | **Notebook precedent**: [8 - Strategies Humaines](./Sudoku-8-HumanStrategies.ipynb) | **Notebook suivant**: [10 - OR-Tools](./Sudoku-10-ORTools.ipynb)